In [1]:
import numpy as np
import pandas as pd
import datascience
from datascience import Table
import folium
from datetime import datetime, timedelta
from datascience.predicates import are
import math

data from https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Map-Year-to-Date-/2fra-mtpn
and http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml

# MISC

In [2]:
def remove_nan(t):
    """Removes all rows with nan values. 
    Note you should use this AFTER stripping the table of columns you do not need"""
    def checkNotnan(val):
        if (val!=val)|(val=='nan')|(val=='NAN')|(val=='NaN'):
            return False
        return True
    for i in range(t.num_columns):
        t = t.where(i, checkNotnan)
    return t

# Taxi Data

In [3]:
df = Table.read_table("green_tripdata_2016-01.csv")

In [4]:
df = df.sample(1000)
#note sampling for ease of use

In [5]:
df = df.select([1,2,5,6,7,8,9])
df.show(5)

lpep_pickup_datetime,Lpep_dropoff_datetime,Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count
2016-01-04 12:49:33,2016-01-04 13:01:18,-73.9911,40.6925,-74.0125,40.6754,1
2016-01-12 07:18:52,2016-01-12 07:28:27,-73.919,40.8163,-73.9372,40.824,5
2016-01-12 12:19:26,2016-01-12 12:25:54,-73.962,40.8103,-73.9525,40.8107,1
2016-01-09 10:45:06,2016-01-09 11:05:29,-73.8618,40.7302,-73.8982,40.7019,1
2016-01-29 20:49:23,2016-01-29 20:53:27,-73.9612,40.8019,-73.9645,40.8071,2


In [6]:
# converting to datetimes

In [7]:
def to_datetime(string_date):
    return datetime.strptime(string_date, '%Y-%m-%d %H:%M:%S')

In [8]:
df_dt = df.with_column('pickup_dt', df.apply(to_datetime, 0)).drop('lpep_pickup_datetime')
df_dt = df_dt.with_column('dropoff_dt', df.apply(to_datetime, 0)).drop('Lpep_dropoff_datetime')

In [9]:
df_dt.show(5)

Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,pickup_dt,dropoff_dt
-73.9911,40.6925,-74.0125,40.6754,1,2016-01-04 12:49:33,2016-01-04 12:49:33
-73.919,40.8163,-73.9372,40.824,5,2016-01-12 07:18:52,2016-01-12 07:18:52
-73.962,40.8103,-73.9525,40.8107,1,2016-01-12 12:19:26,2016-01-12 12:19:26
-73.8618,40.7302,-73.8982,40.7019,1,2016-01-09 10:45:06,2016-01-09 10:45:06
-73.9612,40.8019,-73.9645,40.8071,2,2016-01-29 20:49:23,2016-01-29 20:49:23


In [10]:
df_dt = df_dt.where(0, are.not_equal_to(0))

In [11]:
df_dt.show(5)

Pickup_longitude,Pickup_latitude,Dropoff_longitude,Dropoff_latitude,Passenger_count,pickup_dt,dropoff_dt
-73.9911,40.6925,-74.0125,40.6754,1,2016-01-04 12:49:33,2016-01-04 12:49:33
-73.919,40.8163,-73.9372,40.824,5,2016-01-12 07:18:52,2016-01-12 07:18:52
-73.962,40.8103,-73.9525,40.8107,1,2016-01-12 12:19:26,2016-01-12 12:19:26
-73.8618,40.7302,-73.8982,40.7019,1,2016-01-09 10:45:06,2016-01-09 10:45:06
-73.9612,40.8019,-73.9645,40.8071,2,2016-01-29 20:49:23,2016-01-29 20:49:23


In [12]:
df_loc_pickup = df_dt.select([1,0])
df_loc_dropoff = df_dt.select([3,2])

In [13]:
df_loc_dropoff.row(0)

Row(Dropoff_latitude=40.675380706787109, Dropoff_longitude=-74.012535095214844)

# Complaints Data

In [14]:
complaints = Table.read_table("NY_complaints.csv")

In [15]:
complaints.show(5)

CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,JURIS_DESC,BORO_NM,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
845348933,03/31/2017,23:30:00,nan,nan,03/31/2017,578,HARRASSMENT 2,638,"HARASSMENT,SUBD 3,4,5",COMPLETED,VIOLATION,N.Y. POLICE DEPT,BROOKLYN,69,INSIDE,RESIDENCE - APT. HOUSE,nan,nan,1.01242e+06,171737,40.638,-73.8985,"(40.638018389, -73.898491201)"
886921338,03/31/2017,23:25:00,03/31/2017,23:30:00,03/31/2017,344,ASSAULT 3 & RELATED OFFENSES,101,ASSAULT 3,COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,MANHATTAN,14,nan,STREET,nan,nan,987466,215861,40.7592,-73.9884,"(40.759172699, -73.988392793)"
893265998,03/31/2017,23:15:00,03/31/2017,23:25:00,03/31/2017,105,ROBBERY,394,"ROBBERY,LICENSED FOR HIRE VEHICLE",COMPLETED,FELONY,N.Y. POLICE DEPT,BRONX,42,FRONT OF,TAXI (LIVERY LICENSED),nan,nan,1.0105e+06,245411,40.8402,-73.9051,"(40.84024096, -73.905125257)"
518511851,03/31/2017,23:00:00,03/31/2017,23:10:00,03/31/2017,364,OTHER STATE LAWS (NON PENAL LA,809,TAX LAW,COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,BRONX,49,INSIDE,GROCERY/BODEGA,nan,nan,1.02362e+06,253318,40.8619,-73.8577,"(40.861894559, -73.85766248)"
541009476,03/31/2017,22:55:00,03/31/2017,22:59:00,03/31/2017,235,DANGEROUS DRUGS,511,"CONTROLLED SUBSTANCE, POSSESSI",COMPLETED,MISDEMEANOR,N.Y. POLICE DEPT,BROOKLYN,68,FRONT OF,STREET,nan,nan,977956,167273,40.6258,-74.0227,"(40.625808217, -74.022675222)"


In [16]:
#converting to datetime
def to_datetime_complaints(string_date, string_time):
    return datetime.combine(datetime.strptime(string_date, '%m/%d/%Y'), datetime.strptime(string_time, '%H:%M:%S').time())

In [17]:
complaints_dt = complaints.apply(to_datetime_complaints, [1,2])

/Users/jettjz/anaconda/lib/python3.5/site-packages/datascience/tables.py:362: FutureWarning: column lists are deprecated; pass each as an argument
  "column lists are deprecated; pass each as an argument", FutureWarning)


In [18]:
complaints = complaints.select(['OFNS_DESC','PD_DESC','LAW_CAT_CD','BORO_NM','Longitude','Latitude']).with_column('TIME',complaints_dt)

In [19]:
complaints.show(5)

OFNS_DESC,PD_DESC,LAW_CAT_CD,BORO_NM,Longitude,Latitude,TIME
HARRASSMENT 2,"HARASSMENT,SUBD 3,4,5",VIOLATION,BROOKLYN,-73.8985,40.638,2017-03-31 23:30:00
ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,MISDEMEANOR,MANHATTAN,-73.9884,40.7592,2017-03-31 23:25:00
ROBBERY,"ROBBERY,LICENSED FOR HIRE VEHICLE",FELONY,BRONX,-73.9051,40.8402,2017-03-31 23:15:00
OTHER STATE LAWS (NON PENAL LA,TAX LAW,MISDEMEANOR,BRONX,-73.8577,40.8619,2017-03-31 23:00:00
DANGEROUS DRUGS,"CONTROLLED SUBSTANCE, POSSESSI",MISDEMEANOR,BROOKLYN,-74.0227,40.6258,2017-03-31 22:55:00


In [20]:
#we restrict ourselves to january 2016 from taxi data. potentially can add more but for now this is fine
def in_jan_2016(dt_val):
    start = datetime(2016,1,1,0,0,0)
    end = datetime(2016,2,1,0,0,0)
    return (start<=dt_val)&(dt_val<end)
complaints = complaints.where('TIME', in_jan_2016)
complaints.show(5)

OFNS_DESC,PD_DESC,LAW_CAT_CD,BORO_NM,Longitude,Latitude,TIME
FRAUDS,"FRAUD,UNCLASSIFIED-MISDEMEANOR",MISDEMEANOR,QUEENS,-73.7408,40.6536,2016-01-20 08:00:00
PETIT LARCENY,"LARCENY,PETIT FROM BUILDING,UN",MISDEMEANOR,BRONX,-73.8586,40.8881,2016-01-19 06:00:00
FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",FELONY,MANHATTAN,-73.979,40.7601,2016-01-07 16:27:00
FRAUDS,"FRAUD,UNCLASSIFIED-MISDEMEANOR",MISDEMEANOR,MANHATTAN,-73.979,40.7601,2016-01-07 16:27:00
GRAND LARCENY,"LARCENY,GRAND BY DISHONEST EMP",FELONY,MANHATTAN,-73.988,40.7623,2016-01-04 09:00:00


In [21]:
def non_drug_felony(val):
    if ("DRUG" in val)|("SUBSTANCE" in val):
        return False
    return True

In [22]:
def contains_assault(val):
    if ("ASSAULT" in val):
        return True
    return False

In [23]:
complaints_felony = complaints.where('LAW_CAT_CD', "FELONY").where('PD_DESC', non_drug_felony)
complaints_felony.show(5)

OFNS_DESC,PD_DESC,LAW_CAT_CD,BORO_NM,Longitude,Latitude,TIME
FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",FELONY,MANHATTAN,-73.979,40.7601,2016-01-07 16:27:00
GRAND LARCENY,"LARCENY,GRAND BY DISHONEST EMP",FELONY,MANHATTAN,-73.988,40.7623,2016-01-04 09:00:00
THEFT-FRAUD,"FRAUD,UNCLASSIFIED-FELONY",FELONY,BRONX,-73.8782,40.8749,2016-01-01 12:00:00
THEFT-FRAUD,"FRAUD,UNCLASSIFIED-FELONY",FELONY,BROOKLYN,-73.9598,40.6068,2016-01-01 00:00:00
RAPE,RAPE 1,FELONY,QUEENS,nan,nan,2016-01-01 00:01:00


In [24]:
complaints_felony = remove_nan(complaints_felony)

In [25]:
complaints_felony.show(5)

OFNS_DESC,PD_DESC,LAW_CAT_CD,BORO_NM,Longitude,Latitude,TIME
FORGERY,"FORGERY,ETC.,UNCLASSIFIED-FELO",FELONY,MANHATTAN,-73.979,40.7601,2016-01-07 16:27:00
GRAND LARCENY,"LARCENY,GRAND BY DISHONEST EMP",FELONY,MANHATTAN,-73.988,40.7623,2016-01-04 09:00:00
THEFT-FRAUD,"FRAUD,UNCLASSIFIED-FELONY",FELONY,BRONX,-73.8782,40.8749,2016-01-01 12:00:00
THEFT-FRAUD,"FRAUD,UNCLASSIFIED-FELONY",FELONY,BROOKLYN,-73.9598,40.6068,2016-01-01 00:00:00
NYS LAWS-UNCLASSIFIED FELONY,"NY STATE LAWS,UNCLASSIFIED FEL",FELONY,BROOKLYN,-73.9216,40.6665,2016-01-16 00:01:00


# Visualization

In [26]:
def addMarkers(fol_map, markers, color="blue",icon='star'):
    """adds markers to fol_map based off of table of lat/lon from markers, assumes latitude first then longitude
    limits to 50 markers added at a time to not overload"""
    for i in range(markers.num_rows):
        row = markers.row(i)
        lat = row[0]
        lon = row[1]
        folium.Marker([lat,lon],icon=folium.Icon(color=color, icon=icon)).add_to(fol_map)
        if (i>50):
            return

In [27]:
map_test = folium.Map(location=[40.7128,-74.0059],tiles='Stamen Toner')

In [28]:
addMarkers(map_test, df_loc_dropoff)
addMarkers(map_test, complaints_felony.select(['Latitude','Longitude']), "red")

In [29]:
map_test

# Calculating distance

In [30]:
#could import library but worry about implementing this in class
def dist_coord(lat1,lon1,lat2,lon2):
    '''returns distance in km between to coordinates'''
    '''can handle latitudes/longitudes of same size'''
    R = 6373.0
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (np.sin(dlat/2))**2 + np.cos(lat1) * np.cos(lat2) * (np.sin(dlon/2))**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c
    return distance


In [31]:
def find_close(lat, lon, dist, tab, tablelat=0,tablelon=1):
    """given a table of locations with lat/longitude data, 
    returns table with only rows of distance <= dist to given lat,lon"""
    distances = dist_coord(lat,lon,tab.column(tablelat),tab.column(tablelon))
    out = tab.with_column('DistanceFrom', distances)
    return out.where('DistanceFrom',lambda x:x<dist)

In [32]:
complaints_felony.row(0)

Row(OFNS_DESC='FORGERY', PD_DESC='FORGERY,ETC.,UNCLASSIFIED-FELO', LAW_CAT_CD='FELONY', BORO_NM='MANHATTAN', Longitude=-73.978982121000001, Latitude=40.760115558999999, TIME=datetime.datetime(2016, 1, 7, 16, 27))

In [33]:
lat = complaints_felony.row(5)[5]
lon = complaints_felony.row(5)[4]
test_close = find_close(lat,lon,1,df_dt,3,2)

In [34]:
test_close.select(3,2)

Dropoff_latitude,Dropoff_longitude


In [35]:
map_test_close = folium.Map(location=[40.7128,-74.0059],tiles='Stamen Toner')
addMarkers(map_test_close, test_close.select(3,2))
addMarkers(map_test_close, Table().with_columns('Latitude',lat,'Longitude',lon), "red")
addMarkers(map_test_close, test_close.select(1,0), "green")

In [36]:
map_test_close

In [37]:
#40.853784, -73.877685

# Creating fake data

In [38]:
def make_trip_data(from_lat,from_lon,to_lat,to_lon,error_from,error_to,pickup_dt,dropoff_dt,error_dt):
    '''Creates a trip data with error. returns row'''
    def random_date_uniform(dt,err):
        random_timedelta = timedelta(seconds=np.random.uniform(0, err.total_seconds()))
        return dt + (random_timedelta - err/2)
    pickup_lat = np.random.uniform(from_lat-(error_from/2),from_lat+(error_from/2))
    pickup_lon = np.random.uniform(from_lon-(error_from/2),from_lon+(error_from/2))
    dropoff_lat = np.random.uniform(to_lat-(error_to/2),to_lat+(error_to/2))
    dropoff_lon = np.random.uniform(to_lon-(error_to/2),to_lon+(error_to/2))
    pickup_time = random_date_uniform(pickup_dt, error_dt)
    dropoff_time = random_date_uniform(dropoff_dt, error_dt)
    pickup_time = pickup_time.replace(second=0, microsecond=0)
    dropoff_time = dropoff_time.replace(second=0, microsecond=0)
    out_table = Table().with_columns('Pickup_latitude',pickup_lat,'Pickup_longitude',pickup_lon,
                                     'Dropoff_latitude',dropoff_lat,'Dropoff_longitude',dropoff_lon,
                                    'Passenger_count',1,'Pickup_dt',pickup_time,'Dropoff_dt',dropoff_time)
    return out_table

In [39]:
complaints_felony.row(5)

Row(OFNS_DESC='CRIMINAL MISCHIEF & RELATED OF', PD_DESC='TAMPERING 1,CRIMINAL', LAW_CAT_CD='FELONY', BORO_NM='BRONX', Longitude=-73.897730961000008, Latitude=40.861357898999998, TIME=datetime.datetime(2016, 1, 8, 9, 52))

In [40]:
# 1 km = 0.008983 coords
from_lat = 40.853784
from_lon = -73.877685
to_lon = lon
to_lat = lat
err_f = 0.008983/10
err_t = 0.008983/10
pickup_dt = complaints_felony.column('TIME')[5] - timedelta(seconds=10*60)
dropoff_dt = complaints_felony.column('TIME')[5] + timedelta(seconds=10*60)
err_dt = timedelta(seconds=5*60)

In [41]:
test_synth_row = make_trip_data(from_lat,from_lon,to_lat,to_lon,err_f,err_t,pickup_dt,dropoff_dt,err_dt)

In [42]:
table_test_synth = test_synth_row
for i in range(10):
    table_test_synth = table_test_synth.with_row(make_trip_data(from_lat,from_lon,to_lat,to_lon,err_f,err_t,pickup_dt,dropoff_dt,err_dt))

In [57]:
def make_synth_table(crimes,lat_index,lon_index,dt_index,source_lat,source_lon,distance_err,prob):
    '''returns synthetic data give source_array info and crime table
        goes through each row and with probability given 
        will create a synthetic row corresponding to crime
        TODO: implement time (currently simplified to 10 minutes with 5 min err)'''
    out = None
    for i in range(crimes.num_rows):
        to_lat = crimes.row(i)[lat_index]
        to_lon = crimes.row(i)[lon_index]
        err_f = distance_err
        err_t = distance_err
        pickup_dt = crimes.column(dt_index)[i] - timedelta(seconds=10*60)
        dropoff_dt = crimes.column(dt_index)[i] - timedelta(seconds=10*60)
        err_dt = timedelta(seconds=5*60)
        if (np.random.uniform(0,1) > prob):
            continue
        elif (out==None):
            out = make_trip_data(from_lat,from_lon,to_lat,to_lon,err_f,err_t,pickup_dt,dropoff_dt,err_dt).with_column('Crime Num',i)
        else:
            out = out.with_row(make_trip_data(from_lat,from_lon,to_lat,to_lon,err_f,err_t,pickup_dt,dropoff_dt,err_dt).with_column('Crime Num',i))
            
    return out

In [58]:
source_lat = 40.853784
source_lon = -73.877685
err = 0.008983/10

synth_table_all = make_synth_table(complaints_felony,5,4,6,source_lat,source_lon,err,.8)

In [59]:
synth_table_all.show(5)

Pickup_latitude,Pickup_longitude,Dropoff_latitude,Dropoff_longitude,Passenger_count,Pickup_dt,Dropoff_dt,Crime Num
40.854,-73.8776,40.7599,-73.9791,1,2016-01-07 16:15:00,2016-01-07 16:18:00,0
40.8539,-73.8779,40.7623,-73.9882,1,2016-01-04 08:51:00,2016-01-04 08:48:00,1
40.8539,-73.8778,40.6665,-73.9218,1,2016-01-15 23:49:00,2016-01-15 23:49:00,4
40.8542,-73.8781,40.8617,-73.8973,1,2016-01-08 09:43:00,2016-01-08 09:41:00,5
40.8542,-73.8779,40.837,-73.9146,1,2016-01-01 08:48:00,2016-01-01 08:47:00,6


In [61]:
map_test_synth = folium.Map(location=[40.7128,-74.0059],tiles='Stamen Toner')
addMarkers(map_test_synth, complaints_felony.select(5,4), "red")
addMarkers(map_test_synth, synth_table_all.select(2,3), "orange")
addMarkers(map_test_synth, synth_table_all.select(0,1), "green")

In [62]:
map_test_synth

In [63]:
#TODO: implement a better time creation function based on distance
# also have a linear regression thing to calculate all data
# maybe have multiple superheroes (less batman more defenders)
# move all functions to a .py file
# implement clicking on pins in folium maps to pop up table/row number
# Why am I using Stamen Tower map color is there a cooler one?

#create lab and sample data so as not to have >800 mb preferably
# to be able to run on datahub servers